In [53]:
from statsmodels.tsa.arima_model import ARMA
from statsmodels.graphics.tsaplots import plot_acf
import pandas_datareader.data as web
import pandas
import numpy
import statsmodels.api as sm

In [61]:
prices = pandas.read_csv("prices.csv",parse_dates=['Date'],index_col=0)
tickers = prices.columns[:-2]
# use the last record for each week
prices = prices.resample('W').agg(lambda x:x[-1])
prices.dropna(axis=0, how='any', inplace=True)
rf = prices['^TNX'].values[:-1]
rf /= (52*100)
returns =  prices.iloc[:,:-1].pct_change()[1:]
rm = returns['^GSPC'].values
ri = returns.iloc[:,:-1].values
Ri = ri-rf[:,numpy.newaxis]
Rm = rm-rf
model = sm.OLS(Ri, sm.add_constant(Rm))
results = model.fit()
alpha,beta = results.params
epsilon = numpy.sqrt(Ri.var(axis=0) - beta**2*Rm.var(axis=0))
output = pandas.DataFrame(
    columns=['alpha','beta','epsilon'],
    index = tickers,
    data=numpy.array([alpha,beta,epsilon]).T
)
output.to_csv("coefficients.csv")

In [74]:
from arch.univariate import ARX, GARCH
arx = ARX(rm, lags=1)
arx.volatility = GARCH()
res = arx.fit(disp='off')
pandas.DataFrame(res.params).to_csv("parameters.csv")

  NIT    FC           OBJFUN            GNORM
    1     7    -4.290504E+02     1.636250E+05
    2    21    -4.296904E+02     1.812047E+05
    4    33    -4.296913E+02     1.815156E+05
    5    46    -4.296917E+02     1.812930E+05
    7    58    -4.296948E+02     1.818245E+05
   10    71    -4.296977E+02     1.813353E+05
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -429.69772263056046
            Iterations: 10
            Function evaluations: 71
            Gradient evaluations: 6
